
## My test data is **Dated**!


### This example will increment all the datetime fields in the Hallux database using 

* sqlalchemy to retrieve the data into a dataframe

* a function that will increment a datetime value based on a specific number of years based on a specific calendar date, a day of the week, or holiday

* pymssql to update the database using dynamic sql




### The example starts by importing the required libraries and defining dictionaries 


In [1]:
from os import getenv
import pandas as pd
from datetime import *; from dateutil.relativedelta import *
import calendar

import pymssql
from sqlalchemy import create_engine



# Dictionary of date columns in Hallux with a list for the value (table, artificial key column, increment method) 

Hallux_Dates = {
    
    "Hire_Date"       : ("Agent"         , "Agent_Id"     , "INCREMENT"),
    "Release_Date"    : ("Album"         , "Album_Id"     , "INCREMENT"),
    "Formation_Date"  : ("Band"          , "Band_Id"      , "INCREMENT"),
    "Join_Date"       : ("Band_Member"   , "Member_Id"    , "INCREMENT"),
    "Begin_Date"      : ("Contract"      , "Contract_Id"  , "INCREMENT"),
    "End_Date"        : ("Contract"      , "Contract_Id"  , "INCREMENT"),
    "Order_Date"      : ("Order_Header"  , "Order_Id"     , "INCREMENT"),
    "Promise_Date"    : ("Order_Header"  , "Order_Id"     , "INCREMENT"),
    "Birthdate"       : ("Customer_Profile","Profile_Id"  , "INCREMENT"),
    "Performance_Date": ("Performance"   ,"Performance_Id", "DAYOFWEEK")
}



# holiday dictionary with a list for the value  (month, day of week, earliest day, latest day) 

Holidays = {
    "New Year's Day" : (1,-1,1,1), 
    "Martin Luther King Day" : (1,0,15,21),
    "Washington's Birthday" : (5,0,15,21),
    "Memorial Day" : (5,0,25,31),
    "Juneteenth National Independence Day" : (6,-1,19,19),
    "Independence Day" : (7,-1,4,4),
    "Labor Day" : (9,-1,1,7),
    "Columbus Day" : (10,0,8,14),
    "Halloween" : (10 ,-1,31,31),
    "Veteran's Day" : (11,-1,11,11),
    "Thanksgiving" : (11,3,22,28),
    "Christmas Eve" : (12,-1,24,24),
    "Christmas Day" : (12,-1,25,25),
    "New Year’s Eve" : (12,-1,31,31)
}



### The is_holiday function will determine if a specified date is a holiday

The trick is that some holidays, like the 4th of July, are allows on the same calendar date.  But other holidays, like Thanksgiving, are on a specific weekday and week of the month.  

*This function can be improved by using the Holidays dictionary.*  


In [2]:
# determine if a date is considered a holiday

def is_holiday ( a_date ):   

    
    ret_val = 'No'

    s_date = str(a_date)
    s_date = s_date[0:10]
    x_date = datetime.strptime(s_date, '%Y-%m-%d')
    
    my_month     = x_date.month
    my_day       = x_date.day
    my_weekday   = x_date.weekday()
    
    
    # Exercise: change this logic to loop thru the Holidays dictionary
    
    
    if my_month == 1 and my_day == 1 :
        ret_val = "New Year's Day"
        
    # (3rd Monday in January)
    if my_month == 1 and my_weekday == 0 and my_day >= 15 and my_day <= 21 :
        ret_val = "Martin Luther King Day"
    
    # (3rd Monday in February)
    if my_month == 2 and my_weekday == 0 and my_day >= 15 and my_day <= 21 :
        ret_val = "Washington's Birthday"
        
    # (Last Monday in May)
    if my_month == 5 and my_weekday == 0 and my_day >= 25 and my_day <= 31 :
        ret_val = "Memorial Day"
    
    if my_month == 6 and my_day == 19 :
        ret_val = "Juneteenth National Independence Day"
        
    if my_month == 7 and my_day == 4 :
        ret_val = "Independence Day"
   
    # (First Monday in September)
    if my_month == 9 and my_weekday == 0 and my_day >= 1 and my_day <= 7 :
        ret_val = "Labor Day"
        
    # (Second Monday in October)
    if my_month == 10 and my_weekday == 0 and my_day >= 8 and my_day <= 14 :
        ret_val = "Columbus Day"
        
    if my_month == 10 and my_day == 31 :
        ret_val = "Halloween"
        
    if my_month == 11 and my_day == 11 :
        ret_val = "Veteran's Day"
        
    # (Fourth Thursday in October)
    if my_month == 11 and my_weekday == 3 and my_day >= 22 and my_day <= 28 :
        ret_val = "Thanksgiving"
        
    if my_month == 12 :
        
        if my_day == 24 : 
            ret_val = "Christmas Eve"
        if my_day == 25 : 
            ret_val = "Christmas Day"
        if my_day == 31 : 
            ret_val = "New Year’s Eve"
    
    return ret_val


### The increment_date function takes a table and column name and increments the field by the specified number of years.

This function can update the field by calendar date (INCREMENT) or by the day of the week (DAYOFWEEK).

There are three possible outputs:

* PREVIEW - Display the current date and the new incremented date

* SQL - Generate the sql that would update the database

* UPDATE - update the database directly using dynamic sql


The idea behind the DAYOFWEEK option is that when incrementing performance dates, a concert that was held on a Saturday should still be performed on a Saturday when the date is incremented.  The exception is when a performance is on a fixed holiday like the 4th of July or New Year's Eve. 

*Note: To support sales data the time portion of a datetime column is kept the same as the original date.*  


In [3]:
# Retrieve a datetime field from the specified table and column.
# increment the date by the specified number of years
# return the specified output: 
#  the old and new date (PREVIEW)
#  the sql to update the dates (SQL)
#  or actually update the dates (UPDATE)

def increment_date (a_Table, a_Id_Col, a_Date_Col, a_years, a_method, a_output):
    
    ret_val = 0
    
    # Rows to be updated    
    col=[a_Id_Col,'Curr_Date','New_Date']
          
    sql_select = 'SELECT ' + a_Id_Col + ', ' + a_Date_Col + ' as Curr_Date, convert(date,' + a_Date_Col + ') as New_Date FROM ' + a_Table + ' where ' + a_Date_Col + ' is not null' + ' order by ' + a_Id_Col      
    
    df_date = pd.read_sql(sql_select, eng)
    df_date.set_index(a_Id_Col)

    # add specified number of years 
    for i in range(0,a_years):
        df_date['New_Date'] = df_date['New_Date']+relativedelta(years=+1)

    if a_method == 'DAYOFWEEK':
        
        for i in range(len(df_date)):

            #move to the closest day with the same 

            cdate = df_date['Curr_Date'][i]
            ndate = df_date['New_Date'][i]
            
            # .weekday() 0 = Monday, 6 is Sunday
            # note that for payroll and other systems Sunday is considered the first day of the week not monday
            day_offset = cdate.weekday() - ndate.weekday()
            
            # Handle holidays, a concert on a saturday is not the same as a concert on the 4th of July
            
            special_day = is_holiday (cdate)
            
            if special_day in Holidays:
            
                Holiday_details = Holidays[special_day]

                # only adjust a floating holiday
                if Holiday_details[1] != -1 :
                    ndate = ndate + pd.DateOffset(days=day_offset)
                    # floating holidays can miss by 1 week    
                    if ndate.day < Holiday_details[2] :
                        ndate = ndate + pd.DateOffset(days=7)
                    
            else:    
                
                ndate = ndate + pd.DateOffset(days=day_offset)
       
            df_date.at[i,'New_Date'] = ndate


        
        
    # This procedure will show the data frame, output the sql statements, or update the table
    
    if a_output == 'PREVIEW' :
        
        print(df_date)
        
    else :    

        # generate the sql to update the table with the new date

        for i in range(len(df_date)):
        
            did   = df_date[a_Id_Col][i]
            cdate = df_date['Curr_Date'][i]
            # keep same time as the original date
            ndate = str(df_date['New_Date'][i]) + ' ' + str(cdate.time())
            sql_stm  = "Update " + a_Table + " set " + a_Date_Col + " = '" + str(ndate) + "' where " + a_Id_Col + " = " + str(did)
        
            if a_output == 'SQL' :
                print(cdate,sql_stm)           
            
            if a_output == 'UPDATE' :
                cursor.execute(sql_stm)    

        if a_output == 'UPDATE' :
        
            conn.commit()
        
        if ret_val < 0:
            print(a_Table,'Failure')
        else:        
            print(a_Table,str(len(df_date)) + ' rows updated')
        
    
    return ret_val


### Here we go...

* Create a database engine and a separate connection for dynamic sql

* Specify the number of years to increment the dates and the desired output

* Call the increment_date function for the desired table, column combinations


**Fun Fact:** If the dates are incremented in multiples of 4, leap year data will be maintained!

*This function can be improved by adding user prompts for year and output.*  

*Note: Dataframes perform updates by doing a separate delete and insert. Depending on the constraints defined in the database, the delete may fail.  For this reason, dynamic sql was used to update the database with dynamic update statements.*


In [4]:
# main

hallux_svr = getenv("halluxsvr")
hallux_usr = getenv("halluxusr")
hallux_psd = getenv("halluxpsd")
hallux_db  = getenv("halluxdb")


# for use in creating a dataframe
conn_string = f"mssql+pyodbc://{hallux_usr}:{hallux_psd}@{hallux_svr}/{hallux_db}?driver=SQL+Server+Native+Client+11.0&TrustServerCertificate=yes"
eng = create_engine(conn_string, fast_executemany=True)

# used for executing dynamic sql
conn   = pymssql.connect(hallux_svr, hallux_usr, hallux_psd, hallux_db)
cursor = conn.cursor()

# number of years to add to the dates
# Note that by adding a number years that is divisible by 4, leap years dates are maintained.
num_years = 8

# specify the output: PREVIEW, SQL, or UPDATE
#Preview shows the new dates, sql generates the update statements, update actually updates the dates in the database
the_output = 'PREVIEW'

# increment the dates
for curr_hallux_date, curr_value in Hallux_Dates.items():
    ret = increment_date (curr_value[0], curr_value[1], curr_hallux_date, num_years, curr_value[2], the_output )
    print(curr_hallux_date, ret)

conn.close()
eng.dispose()

    Agent_Id  Curr_Date    New_Date
0     200001 2013-01-30  2021-01-30
1     200002 2018-08-21  2026-08-21
2     200003 2017-08-09  2025-08-09
3     200004 2022-12-01  2030-12-01
4     200005 2016-02-11  2024-02-11
5     200006 2015-04-22  2023-04-22
6     200007 2019-01-15  2027-01-15
7     200008 2021-09-26  2029-09-26
8     200009 2017-07-20  2025-07-20
9     200010 2014-02-12  2022-02-12
10    200011 2022-07-21  2030-07-21
11    200012 2013-01-01  2021-01-01
12    200013 2018-12-26  2026-12-26
13    200014 2015-07-21  2023-07-21
14    200015 2018-06-27  2026-06-27
15    200016 2013-01-01  2021-01-01
16    200017 2013-08-01  2021-08-01
17    200018 2021-03-19  2029-03-19
18    200019 2020-03-07  2028-03-07
19    200020 2014-01-30  2022-01-30
20    200021 2015-10-23  2023-10-23
21    200022 2021-08-16  2029-08-16
22    200023 2017-06-09  2025-06-09
23    200024 2018-07-19  2026-07-19
24    200025 2013-01-01  2021-01-01
25    200026 2018-01-14  2026-01-14
26    200027 2018-11-15  202